# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [93]:
import numpy as np
import pandas as pd
from sklearn import model_selection, datasets, linear_model, metrics

### Генерация датасета

In [94]:
iris = datasets.load_iris()

In [95]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [96]:
classifier = linear_model.SGDClassifier(random_state = 0) #используем SGD-классификатор

### Генерация сетки

In [97]:
classifier.get_params().keys() #список доступных гиперпараметров модели

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [98]:
#словарь сетки значений гиперпараметров
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : range(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
} 

In [99]:
cv = model_selection.StratifiedShuffleSplit(test_size = 0.2, random_state = 0) #стратегия кросс-валидации

### Подбор параметров и оценка качества

#### Grid search - полный перебор параметров (оптимально только для небольших выборок и небольших сеток)

In [100]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring='accuracy', cv = cv,
                                       return_train_score=True) #создание объекта GridSearchCV со стратегией кросс-валидации

In [101]:
%%time
grid_cv.fit(train_data, train_labels) #обучение объекта-сетки

Wall time: 3.13 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
       error_score='raise-deprecating',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=0, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'penalty': ['l1', 'l2'], 'max_iter': range(5, 10), 'alpha': array([0.0001 , 0.00032, 0.00055, 0.00078, 0.001  ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [102]:
grid_cv.best_estimator_ #модель с лучшими параметрами

SGDClassifier(alpha=0.000325, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='squared_hinge',
       max_iter=9, n_iter=None, n_iter_no_change=5, n_jobs=None,
       penalty='l1', power_t=0.5, random_state=0, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [103]:
print(grid_cv.best_score_) #оценка accuracy для лучшей модели
print(grid_cv.best_params_) #словарь параметров этой модели

0.9047619047619048
{'alpha': 0.000325, 'loss': 'squared_hinge', 'max_iter': 9, 'penalty': 'l1'}


In [104]:
df = pd.DataFrame(grid_cv.cv_results_)
df[['params', 'mean_test_score']][:10]

,params,mean_test_score
0,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.752381
1,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.652381
2,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.633333
3,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.642857
4,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.776190
5,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.695238
6,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.719048
7,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.633333
8,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.838095
9,"{'alpha': 0.0001, 'loss': 'hinge', 'max_iter':...",0.800000


#### Randomized grid search - неполный перебор параметров (оптимально для больших выборок и сеток)

In [105]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', 
                                                        cv = cv, n_iter = 20, random_state = 0) #создание объекта RandomizedSearchCV со стратегией кросс-валидаци

In [106]:
%%time
randomized_grid_cv.fit(train_data, train_labels) # обучение RandomizedSearchCV сетки

Wall time: 341 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
          error_score='raise-deprecating',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=0, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=None,
          param_distributions={'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'penalty': ['l1', 'l2'], 'max_iter': range(5, 10), 'alpha': array([0.0001 , 0.00032, 0.00055, 0.00078, 0.001  ])},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [107]:
print(randomized_grid_cv.best_score_)  #оценка accuracy для лучшей из проверенных случайных модели
print(randomized_grid_cv.best_params_) #словарь параметров этой модели

0.8666666666666667
{'penalty': 'l1', 'max_iter': 9, 'loss': 'log', 'alpha': 0.00055}
